In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
import re
import string
import os,glob
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import keras
from nltk.corpus import stopwords

In [ ]:
import pandas as pd
excel_file = r'/content/drive/MyDrive/Embeddings/Fully compiled dataset Sentence New.xlsx'
df = pd.read_excel(excel_file)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
df['Type'] = df['Type'].replace(mapping)
df_type_0 = df[df['Type'] == 0]
df_type_1 = df[df['Type'] == 1]

# Shuffle both DataFrames
df_type_0 = df_type_0.sample(frac=1).reset_index(drop=True)
df_type_1 = df_type_1.sample(frac=1).reset_index(drop=True)

# Calculate the length of the longest DataFrame
max_length = max(len(df_type_0), len(df_type_1))

# Create a new DataFrame to hold the alternating rows
randomized_df = pd.DataFrame(columns=df.columns)

# Iterate through both DataFrames and append rows alternately until one of them runs out
for i in range(max_length):
    if i < len(df_type_0):
        randomized_df = randomized_df.append(df_type_0.iloc[i], ignore_index=True)
    if i < len(df_type_1):
        randomized_df = randomized_df.append(df_type_1.iloc[i], ignore_index=True)
mapping = {'Plagiarized': 1, 'Unplagiarized': 0}
randomized_df['Type'] = randomized_df['Type'].replace(mapping)
randomized_df.drop('ID', axis=1, inplace=True)
randomized_df

In [ ]:
randomized_df['Text'] = randomized_df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert text into lower case

randomized_df['Text'] = randomized_df['Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])    # Remove URLs

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="@"))

#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x[0]!="#"))

#dataset_df['Text'] = dataset_df['Text'].str.replace('\d+', '')  #Remove digits

randomized_df['Text'] = randomized_df['Text'].map(lambda x: re.sub(r'\W+', ' ', x)) # Remove Punctuation marks

#stop = stopwords.words('english')
#dataset_df['Text'] = dataset_df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #Remove stop words

print("\nDataset After Text Preprocessing:")
print("=================================================\n")
randomized_df.head(7)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(randomized_df['Text'])

# Create a DataFrame for TF-IDF matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate TF-IDF DataFrame with the 'Type' column
tfidf_df['Type'] = randomized_df['Type'].values

In [ ]:
tfidf_df

,021,10سالہ,11,111,11اگست,11ستمبر,11ستمبر1948میں,12,1294ھ,12جون,...,۳۳۸,۳۳۹۰,۴۰,۴۳,۴۷,۵۹۰۱,۶۳,ﷺنے,ﷺکا,Type
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

# Assuming tfidf_df contains the TF-IDF vectors and labels
X = tfidf_df.drop(columns=['Type'])  # Features
y = tfidf_df['Type']  # Labels

In [ ]:
# Initialize classifiers
logreg = LogisticRegression()
dtree = DecisionTreeClassifier()
rf = RandomForestClassifier()
svm = SVC()
knn = KNeighborsClassifier()
nb = GaussianNB()
lda = LinearDiscriminantAnalysis()
mlp = MLPClassifier(max_iter=1000)  # Adjust max_iter based on convergence
xgb = GradientBoostingClassifier()

In [ ]:
# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('logreg', logreg),
    ('dtree', dtree),
    ('rf', rf),
    ('svm', svm),
    ('knn', knn),
    ('nb', nb),
    ('lda', lda),
    ('mlp', mlp),
    ('xgb', xgb)
], voting='hard')

In [ ]:
# Perform 5-Fold Cross Validation for each classifier
classifiers = {
    'Logistic Regression': logreg,
    'Decision Tree': dtree,
    'Random Forest': rf,
    'SVM': svm,
    'KNN': knn,
    'Naive Bayes': nb,
    'Linear Discriminant Analysis': lda,
    'MLP': mlp,
    'Gradient Boosting': xgb,
    'Voting Classifier': voting_clf
}


In [ ]:
for name, clf in classifiers.items():
    scores = cross_validate(clf, X, y, cv=5, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
    print(f"{name}:")
    print("Accuracy:", scores['test_accuracy'].mean())
    print("Precision:", scores['test_precision_macro'].mean())
    print("Recall:", scores['test_recall_macro'].mean())
    print("F1 Score:", scores['test_f1_macro'].mean())
    print("\n")

Logistic Regression:
Accuracy: 0.7777777777777778
Precision: 0.7793643156020027
Recall: 0.7777777777777778
F1 Score: 0.7774493601712443


Decision Tree:
Accuracy: 0.6805555555555555
Precision: 0.6814853808185285
Recall: 0.6805555555555555
F1 Score: 0.6801623915595882


Random Forest:
Accuracy: 0.7884920634920636
Precision: 0.791200908199785
Recall: 0.7884920634920636
F1 Score: 0.7880323532884326


SVM:
Accuracy: 0.790079365079365
Precision: 0.7944322399346635
Recall: 0.7900793650793652
F1 Score: 0.7893654528156322


KNN:
Accuracy: 0.7226190476190476
Precision: 0.7726853800332699
Recall: 0.7226190476190476
F1 Score: 0.7092386205963712


Naive Bayes:
Accuracy: 0.7773809523809523
Precision: 0.7787748898667716
Recall: 0.7773809523809524
F1 Score: 0.7771050402056258


Linear Discriminant Analysis:
Accuracy: 0.6912698412698413
Precision: 0.6948249111791641
Recall: 0.6912698412698413
F1 Score: 0.6899076034820635


MLP:
Accuracy: 0.7448412698412699
Precision: 0.745899611470453
Recall: 0.744841